In [3]:
from scipy.spatial import distance as dist
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import time



In [6]:
def eye_aspect_ratio(eye):
    """
    Compute the eye aspect ratio (EAR) for a given eye.
    Args:
        eye (array): Coordinates of the eye landmarks.
    Returns:
        float: The computed EAR.
    """
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Paths and Thresholds
eye_aspect_predictor = r"shape_predictor_68_face_landmarks (1).dat"
video_stream_path = "IMG_3300.MOV"
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 3




In [7]:
# Load Dlib's Face Detector and Facial Landmark Predictor
try:
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(eye_aspect_predictor)
except Exception as e:
    print(f"Error loading models: {e}")
    exit(1)

# Indices for Left and Right Eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# Start the Video Stream


In [9]:
try:
    vs = cv2.VideoCapture(video_stream_path)
except Exception as e:
    exit(1)

COUNTER = 0
TOTAL = 0

while True:
    ret, frame = vs.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)

    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # Draw contours around eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        # Check if EAR is below threshold
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            COUNTER = 0

        # Display Blink Count and EAR
        cv2.putText(frame, f"Blinks: {TOTAL}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vs.release()
cv2.destroyAllWindows()


[INFO] Starting video stream...
